In [1]:
import logging
import sys
import re
import json
from spyne import Application, rpc, ServiceBase, Unicode, ComplexModel
from spyne.protocol.soap import Soap11  
from spyne.server.wsgi import WsgiApplication
from spyne.util.wsgi_wrapper import run_twisted 

logging.basicConfig(level=logging.DEBUG)

class DictionnaryItem(ComplexModel):
    __namespace__ = 'information_extraction'
    key = Unicode
    value = Unicode

class ClientInformationExtractionService(ServiceBase):
    @rpc(Unicode, _returns=DictionnaryItem)
    def extraire_informations(ctx, texte):
        # Dictionnaire pour stocker les informations extraites
        informations = {}
        
        # Liste des clés à extraire
        keys = ["Adresse", "Email", "Numéro de Téléphone", "Montant du Prêt Demandé", "Durée du Prêt", "Description de la Propriété", "Revenu Mensuel", "Dépenses Mensuelles"]

        # Utilisation d'expressions régulières pour extraire chaque information
        for key in keys:
            pattern = rf"{key}:\s*(.*?)\s*(?=\b(?:{'|'.join(keys)})|$)"
            match = re.search(pattern, texte, re.DOTALL)
            if match:
                informations[key] = match.group(1).strip()

        # Affichage pour débogage
        print("Informations extraites :", informations)

        # Retourne les informations en JSON
        return DictionnaryItem(key="informations", value=json.dumps(informations, ensure_ascii=False))

application = Application([ClientInformationExtractionService],
                          tns='spyne.examples.hello',
                          in_protocol=Soap11(),
                          out_protocol=Soap11())

if __name__ == '__main__':
    wsgi_app = WsgiApplication(application)
    sys.exit(run_twisted([(wsgi_app, b'ClientInformationExtractionService')], 8000))


DEBUG:spyne.model.complex:Got bases for DictionnaryItem from meta: (<class 'spyne.model.complex.ComplexModel'>,)
DEBUG:spyne.model.complex:Got bases for extraire_informations from meta: (<class 'spyne.model.complex.ComplexModel'>,)
DEBUG:spyne.model.complex:Got bases for extraire_informationsResponse from meta: (<class 'spyne.model.complex.ComplexModel'>,)
INFO:spyne.application:Initializing application {spyne.examples.hello}Application...
DEBUG:spyne.interface._base:populating __main__.ClientInformationExtractionService types...
DEBUG:spyne.interface._base:  enumerating classes for method 'extraire_informations'
DEBUG:spyne.interface._base:    adding class "<class 'spyne.model.complex.extraire_informations'>" for '{spyne.examples.hello}extraire_informations'
DEBUG:spyne.interface._base:    adding extraire_informations.texte = <class 'spyne.model.primitive.string.Unicode'>
DEBUG:spyne.interface._base:    adding class "<class 'spyne.model.primitive.string.Unicode'>" for '{http://www.w3.

Informations extraites : {'Adresse': '123 Rue de la Liberté, 75001 Paris, France', 'Email': 'john.doe@email.com', 'Numéro de Téléphone': '+33 123 456 789', 'Montant du Prêt Demandé': '200000 EUR', 'Durée du Prêt': '20 ans', 'Description de la Propriété': 'Maison à deux étages\navec jardin, située dans un quartier résidentiel calme.', 'Revenu Mensuel': '5000 EUR', 'Dépenses Mensuelles': '3000 EUR'}


DEBUG:spyne.server.wsgi:Add http header 'accept_encoding' = ['identity']
DEBUG:spyne.server.wsgi:Add http header 'content_length' = ['998']
DEBUG:spyne.server.wsgi:Add http header 'host' = ['localhost:8000']
DEBUG:spyne.server.wsgi:Add http header 'user_agent' = ['Python-urllib/3.11']
DEBUG:spyne.server.wsgi:Add http header 'content_type' = ['text/xml; charset=utf-8']
DEBUG:spyne.server.wsgi:Add http header 'soapaction' = ['"extraire_informations"']
DEBUG:spyne.server.wsgi:Add http header 'connection' = ['close']
DEBUG:spyne.protocol.soap.soap11:ValueError: Deserializing from unicode strings with encoding declaration is not supported by lxml.
DEBUG:spyne.protocol._base:<spyne.protocol.soap.soap11.Soap11 object at 0x000001D64EB70610> attrcache size: 2
DEBUG:spyne.protocol._base:<spyne.protocol.soap.soap11.Soap11 object at 0x000001D64EB70610> attrcache size: 2
DEBUG:spyne.protocol._base:<spyne.protocol.soap.soap11.Soap11 object at 0x000001D64EB70610> attrcache size: 2
DEBUG:spyne.protoco

Informations extraites : {}


SystemExit: 

C:\Users\21627\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
